In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

from holisticai.explainability import Explainer

In [2]:
X = pd.read_csv('data/X_orig.csv', index_col=0)
y = np.load('data/y_orig.npy')

In [3]:
# clean up a bit
for c in X.columns:
    if c.endswith("_isBlank"):
        del X[c]   
X["bmi"] = 10000 * X["weight"].values.copy() / (X["height"].values.copy() * X["height"].values.copy())
del X["weight"]
del X["height"]
del X["urine_hematest_isTrace"] # would have no variance in the strain set
del X["SGOT_isBlankbutapplicable"] # would have no variance in the strain set
del X["calcium_isBlankbutapplicable"] # would have no variance in the strain set
del X["uric_acid_isBlankbutapplicable"] # would only have one true value in the train set
del X["urine_hematest_isVerylarge"] # would only have one true value in the train set
del X["total_bilirubin_isBlankbutapplicable"] # would only have one true value in the train set
del X["alkaline_phosphatase_isBlankbutapplicable"] # would only have one true value in the train set
del X["hemoglobin_isUnacceptable"] # redundant with hematocrit_isUnacceptable
rows = np.where(np.invert(np.isnan(X["systolic_blood_pressure"]) | np.isnan(X["bmi"])))[0]
X = X.iloc[rows,:]
y = y[rows]

name_map = {
    "sex_isFemale": "Sex",
    "age": "Age",
    "systolic_blood_pressure": "Systolic blood pressure",
    "bmi": "BMI",
    "white_blood_cells": "White blood cells", # (mg/dL)
    "sedimentation_rate": "Sedimentation rate",
    "serum_albumin": "Blood albumin",
    "alkaline_phosphatase": "Alkaline phosphatase",
    "cholesterol": "Total cholesterol",
    "physical_activity": "Physical activity",
    "hematocrit": "Hematocrit",
    "uric_acid": "Uric acid",
    "red_blood_cells": "Red blood cells",
    "urine_albumin_isNegative": "Albumin present in urine",
    "serum_protein": "Blood protein"
}
mapped_feature_names = list(map(lambda x: name_map.get(x, x), X.columns))

# split by patient id
pids = np.unique(X.index.values)
train_pids,test_pids = train_test_split(pids, random_state=0)
strain_pids,valid_pids = train_test_split(train_pids, random_state=0)

# find the indexes of the samples from the patient ids
train_inds = np.where([p in train_pids for p in X.index.values])[0]
strain_inds = np.where([p in strain_pids for p in X.index.values])[0]
valid_inds = np.where([p in valid_pids for p in X.index.values])[0]
test_inds = np.where([p in test_pids for p in X.index.values])[0]

# create the split datasets
X_train = X.iloc[train_inds,:]
X_strain = X.iloc[strain_inds,:]
X_valid = X.iloc[valid_inds,:]
X_test = X.iloc[test_inds,:]
y_train = y[train_inds]
y_strain = y[strain_inds]
y_valid = y[valid_inds]
y_test = y[test_inds]

# mean impute for linear and deep models
imp = SimpleImputer()
imp.fit(X_strain)
X_strain_imp = imp.transform(X_strain)
X_train_imp = imp.transform(X_train)
X_valid_imp = imp.transform(X_valid)
X_test_imp = imp.transform(X_test)
X_imp = imp.transform(X)

# standardize
scaler = StandardScaler()
scaler.fit(X_strain_imp)
X_strain_imp = scaler.transform(X_strain_imp)
X_train_imp = scaler.transform(X_train_imp)
X_valid_imp = scaler.transform(X_valid_imp)
X_test_imp = scaler.transform(X_test_imp)
X_imp = scaler.transform(X_imp)

In [4]:
import numpy as np
seed = np.random.seed(42) # set seed for reproducibility
# simple preprocessing

model = RandomForestClassifier() # instantiate model
model.fit(X_train_imp, y_train) # fit model

y_pred = model.predict(X_test_imp) # compute predictions

In [5]:
from holisticai.efficacy.metrics import classification_efficacy_metrics

classification_efficacy_metrics(y_test, y_pred) # compute metrics

,Value,Reference
Metric,,
Accuracy,0.817062,1
Balanced Accuracy,0.718757,1
Precision,0.861423,1
Recall,0.905868,1
F1-Score,0.883087,1


In [6]:
# permutation feature importance
explainer = Explainer(based_on='feature_importance',
                      strategy_type='permutation',
                      model_type='binary_classification',
                      model = model, 
                      x = X_train_imp, 
                      y = y_train)

In [7]:
explainer.metrics() # compute metrics

,Value,Reference
Fourth Fifths,0.088608,0
Importance Spread Divergence,2.067808,-
Importance Spread Ratio,0.526758,0
Global Overlap Score [label=1],0.171540,1
Global Overlap Score [label=0],0.270565,1
Global Range Overlap Score [label=1],0.774366,1
Global Range Overlap Score [label=0],0.885408,1
Global Explainability Ease Score,0.892405,1


In [8]:
# surrogate feature importance
explainer = Explainer(based_on='feature_importance',
                      strategy_type='surrogate',
                      model_type='binary_classification',
                      model = model, 
                      x = X_train_imp, 
                      y = y_train)

In [9]:
explainer.metrics() # compute metrics

,Value,Reference
Fourth Fifths,0.000000,0
Importance Spread Divergence,4.104494,-
Importance Spread Ratio,0.060638,0
Global Explainability Ease Score,0.892405,1
Surrogate Efficacy Classification,0.804426,1


In [10]:
# lime feature importance
explainer = Explainer(based_on='feature_importance',
                      strategy_type='lime',
                      model_type='binary_classification',
                      model = model, 
                      x = X_train_imp, 
                      y = y_train)

In [11]:
explainer.metrics() # compute metrics

,Value,Reference
Dataset Spread Stability,0.011207,0
Dataset Spread Mean,0.655956,0
Dataset Spread Ratio,0.997135,0
Features Spread Stability,0.218643,0
Features Spread Mean,0.155626,0
Features Spread Ratio,0.949961,0
